In [58]:
# import packages
import geopandas as gpd
import osmnx as ox
import momepy
import pandana as pdna
import pandas as pd
import numpy as np
import networkx as nx


In [59]:
barriers = gpd.read_file(r"C:\Users\b8008458\Documents\2021_2022\Scratch Space\York\Barriers")

barriers_access = gpd.read_file(r"C:\Users\b8008458\Documents\2021_2022\Scratch Space\York\Accessability Measures\YorkAccessBarriersScored.gpkg")
barriers_access_bike = gpd.read_file(r"C:\Users\b8008458\Documents\2021_2022\Scratch Space\York\Accessability Measures\YorkAccessBarriersScoredBike.gpkg")

route_useage = gpd.read_file(r"C:\Users\b8008458\Documents\2021_2022\Scratch Space\York\Bike Network\bike_network_routeuseage.shp")
betweeness = gpd.read_file(r"C:\Users\b8008458\Documents\2021_2022\Scratch Space\York\Bike Network\bike_network_costs_bc.shp")

IMD = gpd.read_file(r"C:\Users\b8008458\Documents\2021_2022\Scratch Space\York\IMD\YorkIMD.shp")
carownership = "nan"

min_distance_gained = gpd.read_file(r"C:\Users\b8008458\Documents\2021_2022\Scratch Space\York\ODMatrix\YorkODMatrix.shp")

In [60]:
# set crs
barriers = barriers.to_crs(27700)
barriers_access = barriers_access.to_crs(27700)
barriers_access_bike = barriers_access_bike.to_crs(27700)
route_useage = route_useage.to_crs(27700)
betweenness = betweeness.to_crs(27700)
IMD = IMD.to_crs(27700)
#carownership = carownership.to_crs(27700)
min_distance_gained = min_distance_gained.to_crs(27700)

In [61]:
# calculate min distances from OD Matrix

# remove i - i and j - j 
QGIS_OD_Matrix = min_distance_gained[min_distance_gained['origin_id'] != min_distance_gained['destinatio']]

#%%

# calculate minium network distance 
min_network_distances_df = QGIS_OD_Matrix.groupby('origin_id')['total_cost'].min()

#%%
# convert to dataframe
min_network_distances_df = min_network_distances_df.to_frame().reset_index() 

#%%

# join minimum distances to barriers
barriers['origin_id'] = barriers['globalid']

join = barriers.merge(min_network_distances_df, on = 'origin_id', how='left', suffixes=('', '_y'))
join.drop(join.filter(regex='_y$').columns, axis=1, inplace=True)
barriers = join

In [62]:
# get list of current column headers
barriers_columnlist = list(barriers)

In [63]:
# join IMD rank to barrier
join_left_df = barriers.sjoin(IMD, how="left")
barriers_columnlist += ['CarsPerCit']
barriers_columnlist += ['IMD_Rank']
barriers = join_left_df[barriers_columnlist]


In [64]:
# join barrier_access

join = barriers.merge(barriers_access, on = 'globalid', how ='left', suffixes=('','_y'))
join.drop(join.filter(regex='_y$').columns, axis=1, inplace=True)
barriers = join

In [65]:
# join barrier_access_bike

join = barriers.merge(barriers_access_bike, on = 'globalid', how ='left', suffixes=('','_y'))
join.drop(join.filter(regex='_y$').columns, axis=1, inplace=True)
barriers = join

In [66]:
# join route_access

join = barriers.merge(barriers_access, on = 'globalid', how ='left', suffixes=('','_y'))
join.drop(join.filter(regex='_y$').columns, axis=1, inplace=True)
barriers = join

In [67]:
barriers

,globalid,AuditAmend,CriticalDi,OnNCN,Restrictio,Critical_1,Other,Status,CreationDa,Creator,...,PhotoURL,Count_Atta,min_gain,origin_id,total_cost,geometry,CarsPerCit,IMD_Rank,objectid,amenity_diff
0,54ade712-9b6c-4716-96c0-9efb9f62d005,Auditing,0,yes,BollardMultiple,105,None,Too Narrow,2022-06-20,RH_Access_Sustrans_UK,...,None,None,192.430350,54ade712-9b6c-4716-96c0-9efb9f62d005,150.568612,POINT (460281.698 450314.716),0.473555,29335,46543,82
1,f6daa07a-d3e7-4f04-abd9-4cae134e562f,Auditing,0,yes,BollardMultiple,224,None,Acceptable Width,2022-06-20,RH_Access_Sustrans_UK,...,None,None,177.118244,f6daa07a-d3e7-4f04-abd9-4cae134e562f,104.259386,POINT (460387.948 450211.747),0.380858,28550,39803,81
2,49d7398c-c98a-4466-903b-6e05a146eed8,Auditing,0,no,BollardSingle,126,None,Not NCN,2022-06-20,RH_Access_Sustrans_UK,...,None,None,50.991718,49d7398c-c98a-4466-903b-6e05a146eed8,104.499472,POINT (460444.108 450180.389),0.380858,28550,39803,81
3,763d1bb1-363d-4bc6-b6e6-0efdee707937,Auditing,0,yes,Chicane,191,None,Acceptable Width,2022-06-20,RH_Access_Sustrans_UK,...,None,None,35.030644,763d1bb1-363d-4bc6-b6e6-0efdee707937,62.797804,POINT (460437.235 450271.144),0.401709,31136,39803,81
4,66f3469b-9eb5-497f-b0c3-ebb2d8c8c076,Auditing,0,yes,BollardMultiple,144,None,Too Narrow,2022-06-20,RH_Access_Sustrans_UK,...,None,None,183.496445,66f3469b-9eb5-497f-b0c3-ebb2d8c8c076,62.797804,POINT (460496.223 450273.609),0.401709,31136,39803,81
5,69970194-9501-4692-8fbb-e0ccbefa9714,Auditing,0,yes,BollardMultiple,151,None,Acceptable Width,2022-06-20,RH_Access_Sustrans_UK,...,None,None,141.100915,69970194-9501-4692-8fbb-e0ccbefa9714,183.496445,POINT (460949.377 450298.324),0.380858,28550,25619,96
6,1f1be6c4-7beb-4c15-8cc2-96973ee6e4c1,Auditing,0,yes,BollardSingle,200,None,Acceptable Width,2022-06-20,RH_Access_Sustrans_UK,...,None,None,141.100915,1f1be6c4-7beb-4c15-8cc2-96973ee6e4c1,183.496445,POINT (461128.505 450313.682),0.380858,28550,19213,91
7,3b22a61e-5772-4272-81a4-aa4160ad6492,Auditing,0,yes,Cattlegrid,130,other,Too Narrow,2022-06-20,RH_Access_Sustrans_UK,...,None,None,309.720561,3b22a61e-5772-4272-81a4-aa4160ad6492,309.720561,POINT (461391.205 450404.610),0.380858,28550,32024,96
8,f03d5d9e-dd6a-4995-84e1-843deef6cd7e,Auditing,0,yes,Cattlegrid,121,other,Too Narrow,2022-06-20,RH_Access_Sustrans_UK,...,None,None,104.499472,f03d5d9e-dd6a-4995-84e1-843deef6cd7e,298.140497,POINT (461814.200 450564.136),0.086611,31263,31371,34
9,866c3440-d9cd-44ff-abef-5aea510dccdd,Auditing,0,yes,BollardMultiple,308,None,Acceptable Width,2022-06-20,RH_Access_Sustrans_UK,...,None,None,35.030644,866c3440-d9cd-44ff-abef-5aea510dccdd,298.140497,POINT (461890.782 450826.998),0.086611,31263,31371,34


In [56]:
# join route_useage to barrier
join_left_df = barriers.sjoin_nearest(route_useage, how="left")
barriers_columnlist += ['total_pass']
barriers = join_left_df[barriers_columnlist]

NotImplementedError: Currently, only PyGEOS >= 0.10.0 supports `nearest_all`. To use PyGEOS within GeoPandas, you need to install PyGEOS: 'conda install pygeos' or 'pip install pygeos'

In [ ]:
# join betweeness to barrier
join_left_df = barriers.sjoin_nearest(betweeness, how="left")
barriers_columnlist += ['Betweenes']
barriers = join_left_df[barriers_columnlist]